In [1]:
import time
from pymata4 import pymata4
import threading

In [2]:
LED_PIN = 4
Button_pin=2
timerTime = 5 # time in sec
CB_VALUE = 2

In [3]:
board = pymata4.Pymata4(com_port = "COM4")

pymata4:  Version 1.15

Copyright (c) 2020 Alan Yorinks All Rights Reserved.

Opening COM4...

Waiting 4 seconds(arduino_wait) for Arduino devices to reset...
Arduino compatible device found and connected to COM4

Retrieving Arduino Firmware ID...
Arduino Firmware ID: 1.2 FirmataExpress.ino

Retrieving analog map...
Auto-discovery complete. Found 20 Digital Pins and 6 Analog Pins




Button pressed!
Button pressed!


In [4]:
def LED_OFF():
       board.digital_write(LED_PIN, 0)  # Turn LED off



def LED_ON(data):
    
        if  data[CB_VALUE] == 1:  # Button is pressed
            print("Button pressed!")
            board.digital_write(LED_PIN, 1)  # Turn LED on
            timer = threading.Timer(5.0, LED_OFF)
            timer.start()
           
        time.sleep(0.1)  # Short delay to avoid busy waiting


In [5]:
board.set_pin_mode_digital_output(LED_PIN)
board.set_pin_mode_digital_input(Button_pin,callback=LED_ON)